In [ ]:
import tensorflow as tf

imageHeight = 512
imageWidth = 512

In [ ]:
l = tf.keras.layers
def _conv(inputs, filters, kernel_size, strides, padding, bias=False, normalize=True, activation='relu'):
    output = inputs
    padding_str = 'same'
    if padding>0:
        output = l.ZeroPadding2D(padding=padding)(output)
        padding_str = 'valid'
    output = l.Conv2D(filters, kernel_size, strides, padding_str, use_bias=bias, \
                 kernel_initializer='he_normal', \
                 kernel_regularizer=tf.keras.regularizers.l2(l=5e-4))(output)
    if normalize:
        output = l.BatchNormalization(axis=3)(output)
    if activation=='relu':
        output = l.ReLU()(output)
    if activation=='relu6':
        output = l.ReLU(max_value=6)(output)
    if activation=='leaky_relu':
        output = l.LeakyReLU(alpha=0.1)(output)
    return output

def _residual(inputs, filters, strides):
    shortcut = inputs
    num_channels = shortcut.get_shape().as_list()[-1]
    output = _conv(inputs, filters, 3, 2, 1)
    output = _conv(output, filters, 3, 1, 0, False, True, 'linear')
    if num_channels != filters or strides != 1:
        shortcut = _conv(shortcut, filters, 1, strides, 0, False, Ture, 'linear')
    output = l.Add()([output, shortcut])
    output = l.ReLU()(output)
    return output

def HourglassNetwork():
    image = tf.keras.Input(shape=(imageHeight,imageWidth,3))   #512×512×3
    net = _conv(image, 128, 7, 2, 3)    #256×256×128
    net = _residual(net, 256, 2)        #128×128×256
    net = _hourglass(net)
    